In [12]:
import pandas as pd
import numpy as np
import time as time
import requests

In [13]:
# Read the CSV file
sick_df = pd.read_csv('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/all_delistings_since_2020.csv')

# Clean up CIK numbers
sick_df['cik'] = sick_df['cik'].astype(str).str.zfill(10)

# Convert date string to datetime
sick_df['date'] = pd.to_datetime(sick_df['date'])

# Add columns for SEC data
sick_df['ticker'] = ''
sick_df['sic'] = ''
sick_df['sic_description'] = ''
sick_df['is_biotech'] = False
sick_df['biotech_identification_method'] = ''

In [14]:
print(len(sick_df))
display(sick_df.head())

574


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,,,,False,
1,2024-10-21,TELLURIAN INC. /DE/,0000061398,25,,,,False,
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,,,,False,
3,2024-10-11,Investment Managers Series Trust II,0001587982,25,,,,False,
4,2024-10-07,ORANGE,0001038143,25,,,,False,


In [15]:
# Define biotech-related keywords and patterns (expanded)
biotech_keywords = [
    'therapeutics', 'pharma', 'biotech', 'bio', 'health sciences',
    'medical', 'genetics', 'genomics', 'life sciences', 'biologics',
    'biosciences', 'oncology', 'diagnostic', 'biomedical',
    'laboratory', 'clinical', 'research', 'drug', 'therapeutic',
    'cell therapy', 'gene therapy', 'molecular', 'pharmaceutical'
]

In [16]:
# Define comprehensive biotech and related SIC codes with descriptions
biotech_sic_dict = {
    '2833': 'Medicinal Chemicals and Botanical Products',
    '2834': 'Pharmaceutical Preparations',
    '2835': 'In Vitro and In Vivo Diagnostic Substances',
    '2836': 'Biological Products, Except Diagnostic Substances',
    '3826': 'Laboratory Analytical Instruments',
    '3841': 'Surgical and Medical Instruments',
    '3842': 'Orthopedic, Prosthetic, and Surgical Supplies',
    '3843': 'Dental Equipment and Supplies',
    '3844': 'X-Ray Apparatus and Tubes',
    '3845': 'Electromedical and Electrotherapeutic Apparatus',
    '3851': 'Ophthalmic Goods',
    '8071': 'Medical Laboratories',
    '8731': 'Commercial Physical and Biological Research',
    '8732': 'Commercial Nonphysical Research',
    '8733': 'Noncommercial Research Organizations',
    '8734': 'Testing Laboratories',
    '8999': 'Services, Not Elsewhere Classified (including Medical Research)',
    '6324': 'Hospital and Medical Service Plans',
    '5122': 'Drugs, Proprietaries, and Druggists Sundries'
}

biotech_sic_codes = list(biotech_sic_dict.keys())

In [29]:
# Set up SEC API request headers
headers = {
    'User-Agent': 'sanspeursystems matt@sanspeursystems.com',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'data.sec.gov'
}

# Process each company
for idx, row in sick_df.iterrows():
    print(f"Processing {row['company']} ({idx + 1}/{len(sick_df)})")
    
    # Check company name for biotech keywords
    company_name_lower = row['company'].lower()
    matching_keywords = [kw for kw in biotech_keywords if kw in company_name_lower]
    if matching_keywords:
        sick_df.at[idx, 'is_biotech'] = True
        sick_df.at[idx, 'biotech_identification_method'] = f"name_match:{','.join(matching_keywords)}"
    
    # Get SEC data
    cik = row['cik']
    url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            
            # Store basic info
            sick_df.at[idx, 'ticker'] = data.get('tickers', [''])[0]
            sic_code = str(data.get('sic', '')).strip()
            sick_df.at[idx, 'sic'] = sic_code
            sick_df.at[idx, 'sic_description'] = data.get('sicDescription', '') or biotech_sic_dict.get(sic_code, '')
            
            # Check if SIC code matches biotech
            if sic_code in biotech_sic_codes:
                sick_df.at[idx, 'is_biotech'] = True
                current_method = sick_df.at[idx, 'biotech_identification_method']
                if current_method:
                    sick_df.at[idx, 'biotech_identification_method'] = f"{current_method};sic_{sic_code}"
                else:
                    sick_df.at[idx, 'biotech_identification_method'] = f"sic_{sic_code}"
            
            print(f"Company: {row['company']}")
            print(f"SIC: {sic_code} - {sick_df.at[idx, 'sic_description']}")
            print(f"Ticker: {sick_df.at[idx, 'ticker']}")
            print(f"Is Biotech: {sick_df.at[idx, 'is_biotech']}")
            print("---")
            
    except Exception as e:
        print(f"Error for {row['company']}: {str(e)}")
    
    time.sleep(0.1)  # Respect SEC rate limits

# Filter for biotech companies
btech_delisted_df = sick_df[sick_df['is_biotech']].copy()

# Display results
print("\nPotential Biotech Companies Found:")
print(btech_delisted_df[['date', 'company', 'ticker', 'sic', 'sic_description', 'biotech_identification_method']])

# Save results with detailed information
btech_delisted_df.to_csv('biotech_delistings_detailed.csv', index=False)

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total companies processed: {len(sick_df)}")
print(f"Companies identified as biotech: {len(btech_delisted_df)}")
if len(sick_df) > 0:
    print(f"Percentage biotech: {(len(btech_delisted_df) / len(sick_df)) * 100:.1f}%")

# Print identification method breakdown
print("\nIdentification Method Breakdown:")
print(btech_delisted_df['biotech_identification_method'].value_counts())

# Print SIC code distribution for identified biotech companies
print("\nSIC Code Distribution for Identified Biotech Companies:")
sic_distribution = btech_delisted_df['sic'].value_counts()
for sic in sic_distribution.index:
    description = biotech_sic_dict.get(sic, 'Unknown')
    print(f"SIC {sic}: {description} - {sic_distribution[sic]} companies")

Processing Finch Therapeutics Group, Inc. (1/574)
Company: Finch Therapeutics Group, Inc.
SIC: 2836 - Biological Products, (No Diagnostic Substances)
Ticker: FNCH
Is Biotech: True
---
Processing TELLURIAN INC. /DE/ (2/574)
Company: TELLURIAN INC. /DE/
SIC: 1311 - Crude Petroleum & Natural Gas
Ticker: TELZ
Is Biotech: False
---
Processing Guardion Health Sciences, Inc. (3/574)
Company: Guardion Health Sciences, Inc.
SIC: 2834 - Pharmaceutical Preparations
Ticker: GHSI
Is Biotech: True
---
Processing Investment Managers Series Trust II (4/574)
Error for Investment Managers Series Trust II: list index out of range
Processing ORANGE (5/574)
Company: ORANGE
SIC: 4813 - Telephone Communications (No Radiotelephone)
Ticker: FNCTF
Is Biotech: False
---
Processing Renalytix plc (6/574)
Company: Renalytix plc
SIC: 8071 - Services-Medical Laboratories
Ticker: RNLXY
Is Biotech: True
---
Processing Generations Bancorp NY, Inc. (7/574)
Company: Generations Bancorp NY, Inc.
SIC: 6035 - Savings Institu

In [30]:
print(len(btech_delisted_df))
display(btech_delisted_df.head())  

58


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
10,2024-09-13,Calliditas Therapeutics AB,0001795579,25,,,,True,"name_match:therapeutics,therapeutic"


# remember you are dropping the ones you couldny get a ticker symbol for right now

In [31]:
# drop the rows with no ticker. then sort by date and reset index
btech_delisted_df = btech_delisted_df[btech_delisted_df['ticker'] != '']

print(len(btech_delisted_df))  
display(btech_delisted_df.head())

42


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
23,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834


In [35]:
# --
# the empty sicks
# --
btech_delisted_df[btech_delisted_df['ticker'] == '']


,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method


In [32]:
# view again
display(btech_delisted_df.head())

,date,company,cik,form,ticker,sic,sic_description,is_biotech,biotech_identification_method
0,2024-10-31,"Finch Therapeutics Group, Inc.",0001733257,25,FNCH,2836,"Biological Products, (No Diagnostic Substances)",True,"name_match:therapeutics,therapeutic;sic_2836"
2,2024-10-11,"Guardion Health Sciences, Inc.",0001642375,25,GHSI,2834,Pharmaceutical Preparations,True,name_match:health sciences;sic_2834
5,2024-10-07,Renalytix plc,0001811115,25,RNLXY,8071,Services-Medical Laboratories,True,sic_8071;sic_8071;sic_8071
9,2024-09-20,Telesis Bio Inc.,0001850079,25,TBIO,3826,Laboratory Analytical Instruments,True,name_match:bio;sic_3826
23,2024-07-25,MorphoSys AG,0001340243,25,MPSYF,2834,Pharmaceutical Preparations,True,sic_2834;sic_2834


In [33]:
# save to csv 
btech_delisted_df.to_csv('btech_delisted_df-delistings_since_2020.csv', index=False)

get board members from delisted
check for matches with current NEV stocks

?

look back 1 month
look back up to and including current and previous quarter chart data
other technical and chart data
financials
news

?

| END |
|-|
| this all works. |